In [1]:
import torch
from torch.utils.data import DataLoader
from dataset import RTB_old
import torch.nn as nn
import torch.nn.functional as F
from model import ProbabilityANN
import torch.optim as optim
from tqdm import tqdm
import os

imp_file_path = "Adobe_dataset/imp.06.txt"
clk_file_path = "Adobe_dataset/clk.06.txt"
conv_file_path = "Adobe_dataset/conv.06.txt"

train_dataset = RTB_old(imp_file_path,clk_file_path,conv_file_path, split='train')
val_dataset = RTB_old(imp_file_path,clk_file_path,conv_file_path, split='val')

Key 16751 not in column 
Attribute : Long-term interest/sex



/home/somin/.conda/envs/hugging_face/lib/python3.8/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


imp_train shape: torch.Size([1724321, 53])
imp_val shape: torch.Size([90754, 53])
clk_train shape: torch.Size([1724321])
clk_val shape: torch.Size([90754])
conv_train shape: torch.Size([1724321])
conv_val shape: torch.Size([90754])
Key 16751 not in column 
Attribute : Long-term interest/sex

imp_train shape: torch.Size([1724321, 53])
imp_val shape: torch.Size([90754, 53])
clk_train shape: torch.Size([1724321])
clk_val shape: torch.Size([90754])
conv_train shape: torch.Size([1724321])
conv_val shape: torch.Size([90754])


In [4]:
from model import ProbabilityANN
from torchvision.ops.focal_loss import sigmoid_focal_loss
from sklearn.metrics import confusion_matrix

device = "cuda" if torch.cuda.is_available() else "cpu"


batch_size = 10000

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

model = ProbabilityANN().to(device)

weights_folder = f"weights"
os.makedirs(weights_folder, exist_ok=True)

optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 50

In [5]:
for epoch in tqdm(range(epochs), desc="Epoch : "):
    
    running_loss = 0
    progress_bar = tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{epochs}", position=0, leave=True)
    
    for i, data in enumerate(train_loader, 0):
        
        inputs, clk_label, conv_label = data[0].to(device), data[1].to(device), data[2].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
       
        clk_prob , conv_prob = outputs[:,0] , outputs[:,1]
        
        loss1 = sigmoid_focal_loss(clk_prob, clk_label, alpha=0.9993, gamma=2, reduction="mean")
        loss2 = sigmoid_focal_loss(conv_prob, conv_label, alpha=0.9996, gamma=2, reduction="mean")

        loss = loss1 + loss2
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss
        
        progress_bar.set_postfix(running_loss=running_loss / (i+1))
        progress_bar.update(1)
    
        # if(i % ((len(train_dataset)//batch_size)//10) == 0):
        #     print(f"Batch {i+1} | Epoch Loss {running_loss} | Iteration Loss {loss.item()}")
        
        # if(i % ((len(train_dataset)//batch_size)//5) == 0):
        #     weights_file_path = f"{weights_folder}/Epoch_{epoch+1}_Iteration_{i}_loss_{loss}.pt"
        #     torch.save(model.state_dict(), weights_file_path)
        
        # iteration_tqdm.set_postfix(loss=loss.item())
    
    progress_bar.close()    
    
    print(f"Epoch {epoch+1} | Loss {running_loss} ")
    weights_file_path = f"{weights_folder}/Epoch_{epoch+1}_Loss_{running_loss}.pt"
    # torch.save(model.state_dict(), weights_file_path)
    
    threshold = 0.5
    model.eval()
    all_clk_preds, all_clk_labels = [], []
    all_conv_preds, all_conv_labels = [], []
    val_loss = 0
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            inputs, clk_label, conv_label = data[0].to(device), data[1].to(device), data[2].to(device)
            outputs = model(inputs)
            
            clk_prob, conv_prob = outputs[:,0], outputs[:,1]
            
            loss1 = sigmoid_focal_loss(clk_prob, clk_label, alpha=0.9993, gamma=2, reduction="mean")
            loss2 = sigmoid_focal_loss(conv_prob, conv_label, alpha=0.9996, gamma=2, reduction="mean")
            
            clk_pred = (clk_prob > threshold).cpu().numpy()
            conv_pred = (conv_prob > threshold).cpu().numpy()
            
            # Collect ground-truth labels and predictions
            all_clk_preds.extend(clk_pred)
            all_clk_labels.extend(clk_label.cpu().numpy())
            all_conv_preds.extend(conv_pred)
            all_conv_labels.extend(conv_label.cpu().numpy())
            
            loss = loss1 + loss2
            val_loss += loss.item()
    
    val_loss /= len(val_loader)
    print(f"Validation Loss after Epoch {epoch+1}: {val_loss:.4f}") 
    
    clk_conf_matrix = confusion_matrix(all_clk_labels, all_clk_preds)
    conv_conf_matrix = confusion_matrix(all_conv_labels, all_conv_preds)

    print(f"Confusion Matrix for Click:")
    print(clk_conf_matrix)

    print(f"Confusion Matrix for Conversion:")
    print(conv_conf_matrix)       

Epoch 1/50: 100%|██████████| 173/173 [00:26<00:00,  6.50it/s, running_loss=tensor(0.0003, grad_fn=<DivBackward0>)]


Epoch 1 | Loss 0.04645425081253052 


Validation Loss after Epoch 1: 0.0003
Confusion Matrix for Click:
[[90697     0]
 [   57     0]]
Confusion Matrix for Conversion:
[[90751     0]
 [    3     0]]


Epoch 2/50: 100%|██████████| 173/173 [00:25<00:00,  6.75it/s, running_loss=tensor(0.0003, grad_fn=<DivBackward0>)]


Epoch 2 | Loss 0.04449288919568062 


...

Epoch 15/50: 100%|██████████| 173/173 [00:25<00:00,  6.71it/s, running_loss=tensor(0.0002, grad_fn=<DivBackward0>)]


Epoch 15 | Loss 0.04136139899492264 


Validation Loss after Epoch 15: 0.0002
Confusion Matrix for Click:
[[90697     0]
 [   57     0]]
Confusion Matrix for Conversion:
[[90751     0]
 [    3     0]]


Epoch :  30%|███       | 15/50 [07:17<17:00, 29.17s/it]9it/s, running_loss=tensor(0.0002, grad_fn=<DivBackward0>)]


KeyboardInterrupt: 